In [1]:
! pip install semantic-kernel -U
! pip install qdrant_client -U

In [2]:
import os
import json

In [3]:
import semantic_kernel as sk
from dotenv import load_dotenv
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
kernel = sk.Kernel()
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [5]:
kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("GPT3Model",endpoint,api_key=api_key,api_version = "2023-07-01-preview"))


kernel.add_text_embedding_generation_service(
        "embeddings_services", skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-07-01-preview")
)

In [6]:
base_plugin = "./plugins"

In [7]:
files_plugin = kernel.import_semantic_skill_from_directory(base_plugin , "FilePlugin")

In [8]:
nodes_files = os.listdir("./data/notes")
transcrips_files = os.listdir("./data/transcripts")

In [9]:
kblist = []

In [10]:
for f in nodes_files:
    file = open("./data/notes/"+f, "r") 
    content = file.read()
    notesFunc = files_plugin["Notes"]
    result = notesFunc(content)
    json_result = json.loads(result.result)
    kblist.append(json_result)
    file.close()

In [11]:
for f in transcrips_files:
    file = open("./data/transcripts/"+f, "r") 
    content = file.read()
    transcripsFunc = files_plugin["Transcrips"]
    result = transcripsFunc(content)
    json_result = json.loads(result.result)
    for item in json_result:
        kblist.append(item)
    file.close()

In [12]:
kblist

[{'kb': 'History of machine learning',
  'content': 'In this lesson, we will walk through the major milestones in the history of machine learning and artificial intelligence. The history of artificial intelligence (AI) as a field is intertwined with the history of machine learning, as the algorithms and computational advances that underpin ML fed into the development of AI. It is useful to remember that, while these fields as distinct areas of inquiry began to crystallize in the 1950s, important algorithmic, statistical, mathematical, computational and technical discoveries predated and overlapped this era. In fact, people have been thinking about these questions for hundreds of years: this article discusses the historical intellectual underpinnings of the idea of a \'thinking machine.\'\n\nNotable discoveries:\n- 1763, 1812 Bayes Theorem and its predecessors. This theorem and its applications underlie inference, describing the probability of an event occurring based on prior knowledge

In [13]:
base_vectordb = 'aboutMLKBDemoDemo'

In [14]:

from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [15]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection_async(base_vectordb)

In [16]:
kernel.register_memory_store(memory_store=qdrant_store)

In [17]:
import uuid

In [18]:
uuid.uuid4()

UUID('778692a2-56a8-4c37-b19a-74104b23fd41')

In [19]:
for item in kblist:
    content = item["kb"] + ' - ' + item["content"]
    id =str(uuid.uuid4())
    await kernel.memory.save_information_async(base_vectordb, id=id, text=content)

In [20]:
ask = "can you tell me what is different ML and AI"

memories = await kernel.memory.search_async(
    base_vectordb, ask, limit=1, min_relevance_score=0.8
)

In [21]:
result = ''
for memory in memories:
    print(f"Top Result: {memory.text} with score {memory.relevance}")
    result = memory.text

Top Result: The difference between AI and ML - start by talking about the difference between artificial intelligence and machine learning AI is a science of getting machines to accomplish tasks that typically require human level intelligence many different techniques have been proposed for AI but the most successful and popular approach these days is machine learning unlike other AI techniques ml uses specialized algorithms to make decisions by learning from data so machine learning is really a subset of artificial intelligence you've also probably heard of deep learning which is a subset of machine learning that relies on neural networks to learn from data in this course we're with score 0.8928732


In [22]:
answer_plugin = kernel.import_semantic_skill_from_directory(base_plugin , "AnswerPlugin")

In [23]:
answerFunc = answer_plugin["Summary"]

In [24]:
result

"The difference between AI and ML - start by talking about the difference between artificial intelligence and machine learning AI is a science of getting machines to accomplish tasks that typically require human level intelligence many different techniques have been proposed for AI but the most successful and popular approach these days is machine learning unlike other AI techniques ml uses specialized algorithms to make decisions by learning from data so machine learning is really a subset of artificial intelligence you've also probably heard of deep learning which is a subset of machine learning that relies on neural networks to learn from data in this course we're"